In [29]:
# imports
from torch.utils.data import Dataset

from torch import nn

from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
#from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import time
import cv2
import os
import json
from typing import Dict, List

In [ ]:
# Dataset class

class ImageDataset(Dataset):
    def __init__(self, images: Dict, transforms=None) -> None:
        self.images = images
        self.transforms = transforms
    def __getitem__(self, index):
        image = self.tensors[0][index]
        label = self.tensors[1][index]
        bbox = self.tensors[2][index]
        
        image = image.permute(2, 0, 1)
        if self.transforms_
            image = self.transforms(image)

        return (image, label, bbox)
    def __len__(self):
        return self.tensors[0].size(0)


In [ ]:
class ObjectDetector(nn.Module):
    def __init__(self, base_model, num_classes) -> None:
        super().__init__()
        self.base_model = base_model
        self.regressor = nn.Sequential(
            nn.Linear(base_model.fc.in_features, 128), 
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(32, 4),
            nn.Sigmoid()
            
        )
        self.classifier = nn.Sequential(
			nn.Linear(base_model.fc.in_features, 512),
			nn.ReLU(),
			nn.Dropout(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Dropout(),
			nn.Linear(512, self.numClasses)
		)
		# set the classifier of our base model to produce outputs
		# from the last convolution block
        self.base_model.fc = nn.Identity()
    def forward(self, x):
        features = self.base_model(x)
        bboxes = self.regressor(x)
        class_logits = self.classifier(x)
        return (bboxes, class_logits)



In [32]:
# training script

##  Read the annoatitions from the directory
annotations_path = "../../datasets/annotations/"
images_path = "../frames/"
annotations_files = os.listdir("../../datasets/annotations/")
images: List[torch.Tensor] = []
image_bboxes = []
for json_file_name in annotations_files:
    full_annotations_path = annotations_path + json_file_name
    f  = open(full_annotations_path, 'r')
    annotation_dict = json.load(f)[0] # as it is a list with one element the dict
    coordinates: Dict = {
        "boxes": []
    }
    for bbox in annotation_dict["annotations"]:
        coordinates["boxes"].append(bbox["coordinates"])
        #print(coordinates["boxes"])
    image_bboxes.append(coordinates)
    #print(image_bboxes[0])
    image = cv2.imread(images_path + annotation_dict["image"])
    image: torch.Tensor = torch.tensor(image).permute(1,0,2) # (X, Y, RGB) (W,H,RGB) we do this to match with the bboxes coordinates (x,y)
    images.append(image)
    break
    


<class 'list'>
